In [152]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(
    host="pgsql-196447.vipserv.org",
    port=5432, 
    dbname='wbauer_adb',
    user='wbauer_adb', 
    password='adb2020'
    )


Ile kategorii filmów mamy w wypożyczalni?

In [153]:
num_of_categories = pd.read_sql("SELECT COUNT(category_id) FROM category",con=connection)
print(num_of_categories)

   count
0     16


Wyświetl listę kategorii w kolejności alfabetycznej.

In [154]:
categories_names = pd.read_sql("SELECT name FROM category ORDER BY name",con=connection)
categories_names

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


Znajdź najstarszy i najmłodszy film do wypożyczenia.

In [155]:
newest_films = pd.read_sql("""
                            SELECT film_id, title, MAX(release_year) 
                            FROM film GROUP BY film_id
                            """,con=connection)
print(newest_film)

     film_id                  title   min
0        251    Dragonfly Strangers  2006
1        106  Bulworth Commandments  2006
2        120      Caribbean Liberty  2006
3        285       English Bulworth  2006
4        681        Pirates Roxanne  2006
..       ...                    ...   ...
995      616         National Story  2006
996      615          Nash Chocolat  2006
997      144    Chinatown Gladiator  2006
998        7        Airplane Sierra  2006
999      453         Image Princess  2006

[1000 rows x 3 columns]


In [156]:
oldest_films = pd.read_sql('SELECT film_id, title, MIN(release_year) FROM film GROUP BY film_id',con=connection)
print(oldest_films)

     film_id                  title   min
0        251    Dragonfly Strangers  2006
1        106  Bulworth Commandments  2006
2        120      Caribbean Liberty  2006
3        285       English Bulworth  2006
4        681        Pirates Roxanne  2006
..       ...                    ...   ...
995      616         National Story  2006
996      615          Nash Chocolat  2006
997      144    Chinatown Gladiator  2006
998        7        Airplane Sierra  2006
999      453         Image Princess  2006

[1000 rows x 3 columns]


Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?

In [157]:
rentals_1 = pd.read_sql("""
                        SELECT COUNT(rental_id)
                        FROM rental 
                        WHERE rental_date 
                        BETWEEN '2005-07-01 00:00:00' AND '2005-08-01 23:59:59'
                        """
                        ,con=connection)
print(rentals_1)

   count
0   7380


Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?

In [158]:
rentals_2 = pd.read_sql("""
                        SELECT COUNT(rental_id) 
                        FROM rental 
                        WHERE rental_date 
                        BETWEEN '2010-01-01 00:00:00' AND '2011-02-01 23:59:59'
                        """
                        ,con=connection)
print(rentals_2)

   count
0      0


Znajdź największą płatność wypożyczenia.

In [159]:
biggest_payment = pd.read_sql("SELECT MAX(amount) AS max_payment_amount FROM payment",con=connection)
print(biggest_payment)

   max_payment_amount
0               11.99


Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.

In [160]:
customers = pd.read_sql("""
                          SELECT *
                          FROM customer
                          INNER JOIN address
                            ON customer.address_id = address.address_id
                          INNER JOIN city
                            ON address.city_id = city.city_id
                          INNER JOIN country
                            ON city.country_id = country.country_id
                          WHERE country.country IN ('Poland', 'Nigeria', 'Bangladesh')
                        """
                        ,con=connection)
print(customers)

    customer_id  store_id  ...     country         last_update
0           284         1  ...     Nigeria 2006-02-15 09:44:00
1           318         1  ...      Poland 2006-02-15 09:44:00
2           572         1  ...      Poland 2006-02-15 09:44:00
3           409         2  ...     Nigeria 2006-02-15 09:44:00
4           332         1  ...  Bangladesh 2006-02-15 09:44:00
5           286         1  ...     Nigeria 2006-02-15 09:44:00
6            80         1  ...     Nigeria 2006-02-15 09:44:00
7           198         2  ...     Nigeria 2006-02-15 09:44:00
8           103         1  ...     Nigeria 2006-02-15 09:44:00
9           329         2  ...  Bangladesh 2006-02-15 09:44:00
10          128         1  ...      Poland 2006-02-15 09:44:00
11           18         2  ...     Nigeria 2006-02-15 09:44:00
12          270         1  ...      Poland 2006-02-15 09:44:00
13          501         1  ...      Poland 2006-02-15 09:44:00
14          277         2  ...     Nigeria 2006-02-15 0

Gdzie mieszkają członkowie personelu?

In [161]:
staff_countries = pd.read_sql("""
                                SELECT DISTINCT country.country 
                                FROM staff
                                INNER JOIN address
                                  ON staff.address_id = address.address_id
                                INNER JOIN city
                                  ON address.city_id = city.city_id
                                INNER JOIN country
                                  ON city.country_id = country.country_id
                                """
                                
                                ,con=connection)
print(staff_countries)

     country
0  Australia
1     Canada


Ilu pracowników mieszka w Argentynie lub Hiszpanii?

In [162]:
num_staff_from_countries = pd.read_sql("""
                                        SELECT COUNT(staff_id) 
                                        FROM staff
                                        INNER JOIN address
                                          ON staff.address_id = address.address_id
                                        INNER JOIN city
                                          ON address.city_id = city.city_id
                                        INNER JOIN country
                                          ON city.country_id = country.country_id
                                        WHERE country.country IN ('Spain', 'Argentina')
                                        """
                                        
                                        ,con=connection)
print(num_staff_from_countries)


   count
0      0


Jakie kategorie filmów zostały wypożyczone przez klientów?

In [163]:
client_categories = pd.read_sql("""
                                  SELECT DISTINCT category.category_id, category.name
                                  FROM rental
                                  INNER JOIN inventory
                                    ON rental.inventory_id = inventory.inventory_id
                                  INNER JOIN film
                                    ON inventory.film_id = film.film_id
                                  INNER JOIN film_category
                                    ON film.film_id = film_category.film_id
                                  INNER JOIN category
                                    ON film_category.category_id = category.category_id
                                  ORDER BY category.category_id
                                  """
                                  ,con=connection)
print(client_categories)

    category_id         name
0             1       Action
1             2    Animation
2             3     Children
3             4     Classics
4             5       Comedy
5             6  Documentary
6             7        Drama
7             8       Family
8             9      Foreign
9            10        Games
10           11       Horror
11           12        Music
12           13          New
13           14       Sci-Fi
14           15       Sports
15           16       Travel


Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.

In [164]:
america_categories = pd.read_sql("""
                                  SELECT DISTINCT category.category_id, category.name
                                  FROM country
                                  INNER JOIN city
                                    ON country.country in ('United States') AND country.country_id = city.country_id
                                  INNER JOIN address
                                    ON address.city_id = city.city_id
                                  INNER JOIN customer
                                    ON address.address_id = customer.address_id
                                  INNER JOIN rental
                                    ON customer.customer_id = rental.customer_id
                                  INNER JOIN inventory
                                    ON rental.inventory_id = inventory.inventory_id
                                  INNER JOIN film
                                    ON inventory.film_id = film.film_id
                                  INNER JOIN film_category
                                    ON film.film_id = film_category.film_id
                                  INNER JOIN category
                                    ON film_category.category_id = category.category_id
                                  ORDER BY category.category_id
                                  """
                                  ,con=connection)
print(america_categories)

    category_id         name
0             1       Action
1             2    Animation
2             3     Children
3             4     Classics
4             5       Comedy
5             6  Documentary
6             7        Drama
7             8       Family
8             9      Foreign
9            10        Games
10           11       Horror
11           12        Music
12           13          New
13           14       Sci-Fi
14           15       Sports
15           16       Travel


Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley

In [165]:
films_with_actors = pd.read_sql("""
                                  SELECT DISTINCT film.title
                                  FROM film
                                  INNER JOIN film_actor
                                    ON film.film_id = film_actor.film_id
                                  INNER JOIN actor
                                    ON (film_actor.actor_id = actor.actor_id) 
                                  WHERE (
                                        (actor.first_name='Olympia' AND actor.last_name='Pfeiffer')
                                    OR  (actor.first_name='Julia' AND actor.last_name='Zellweger')
                                    OR  (actor.first_name='Ellen' AND actor.last_name='Presley')
                                    )
                                  ORDER BY film.title
                                  
                                  """
                                  ,con=connection)
print(films_with_actors)

                   title
0            Badman Dawn
1        Bilko Anonymous
2   Breakfast Goldfinger
3      Caribbean Liberty
4       Casper Dragonfly
..                   ...
64             Turn Star
65            Wait Cider
66          Women Dorado
67             Won Dares
68         Wyoming Storm

[69 rows x 1 columns]
